# Tutorial de Algoritmos Genéticos: Optimización de Planificación de Tareas

## Introducción

Los Algoritmos Genéticos (AGs) son técnicas de optimización poderosas inspiradas en la evolución natural. Funcionan evolucionando una población de soluciones candidatas a través de múltiples generaciones, usando operadores inspirados en la evolución biológica: selección, cruce (reproducción) y mutación.

En este tutorial, resolveremos un **problema de planificación de tareas con restricciones de precedencia** usando un algoritmo genético. Este es un problema clásico de optimización combinatoria donde necesitamos ordenar tareas respetando dependencias entre ellas..

## El Problema: Planificación de Tareas con Restricciones de Precedencia

Tenemos 10 tareas (A hasta J) que necesitan ser programadas en un orden específico. Sin embargo, algunas tareas tienen **restricciones de precedencia** — solo pueden comenzar después de que ciertas otras tareas se hayan completado.

### Definición del Problema

**Tareas:** A, B, C, D, E, F, G, H, I, J

**Restricciones de Precedencia:**
- La tarea C requiere que A se complete primero
- La tarea D requiere que A se complete primero
- La tarea E requiere que B se complete primero
- La tarea F requiere que tanto C como D se completen primero
- La tarea G requiere que tanto E como F se completen primero
- La tarea H requiere que G se complete primero
- La tarea I requiere que F se complete primero
- La tarea J requiere que tanto H como I se completen primero

![DAG](figuras/figura1_grafo_precedencias.png)

Nuestro objetivo es encontrar un ordenamiento válido de estas tareas que:
1. Respete todas las restricciones de precedencia
2. Minimice la "distancia" entre tareas dependientes (manteniendo tareas relacionadas cerca)

## Parte 1: Configuración del Problema

Comencemos definiendo nuestro problema en código y creando las funciones principales.

In [17]:
# Importar biblioteca requerida
import random  # Para generar números aleatorios y mezclar

In [18]:
# --- Datos del problema ---
# Lista de todas las tareas que necesitamos programar
tasks = ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J']
precedence = {
    'C': ['A'],        # C requiere A
    'D': ['A'],        # D requiere A
    'E': ['B'],        # E requiere B
    'F': ['C', 'D'],   # F requiere tanto C como D
    'G': ['E', 'F'],   # G requiere tanto E como F
    'H': ['G'],        # H requiere G
    'I': ['F'],        # I requiere F
    'J': ['H', 'I']    # J requiere tanto H como I
}

print("Tareas:", tasks)
print("\nRestricciones de Precedencia:")
for task, deps in precedence.items():
    print(f"  {task} requiere: {deps}")


Tareas: ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J']

Restricciones de Precedencia:
  C requiere: ['A']
  D requiere: ['A']
  E requiere: ['B']
  F requiere: ['C', 'D']
  G requiere: ['E', 'F']
  H requiere: ['G']
  I requiere: ['F']
  J requiere: ['H', 'I']


### Función de Validación

Necesitamos una forma de verificar si un orden de tareas es válido (respeta todas las restricciones):

In [19]:
def is_valid(order):
    # Crear un diccionario que mapea cada tarea a su posición en el orden
    # Ejemplo: si order = ['B', 'A', 'C'], entonces position = {'B': 0, 'A': 1, 'C': 2}
    position = {task: i for i, task in enumerate(order)}
    
    # Iterar sobre cada tarea que tiene restricciones de precedencia
    for task, preds in precedence.items():
        # Para cada predecesor requerido por esta tarea
        for pred in preds:
            # Verificar si el predecesor viene DESPUÉS de la tarea (posición mayor)
            # Si es así, la restricción está violada
            if position[pred] > position[task]:
                return False  # Orden inválido
    
    # Si llegamos aquí, todas las restricciones están satisfechas
    return True

# Probar la función de validación
valid_order = ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J']
invalid_order = ['C', 'A', 'B', 'D', 'E', 'F', 'G', 'H', 'I', 'J']

print(f"Orden {valid_order}")
print(f"  Válido: {is_valid(valid_order)}\n")

print(f"Orden {invalid_order}")
print(f"  Válido: {is_valid(invalid_order)} (C viene antes que A, violando la restricción)")

Orden ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J']
  Válido: True

Orden ['C', 'A', 'B', 'D', 'E', 'F', 'G', 'H', 'I', 'J']
  Válido: False (C viene antes que A, violando la restricción)


### Función de Aptitud (Fitness)

La función de aptitud evalúa qué tan buena es una solución. Mayor aptitud = mejor solución.

**Entendiendo la Función de Aptitud:**
- Puntaje base: 100 puntos
- Restricción violada: -30 puntos (hace la solución inválida)
- Penalización por distancia: -2 puntos por cada tarea entre un predecesor y su tarea dependiente
- Esto fomenta soluciones válidas donde las tareas relacionadas están programadas cerca

In [20]:
def fitness(order):
    # Crear un diccionario que mapea cada tarea a su posición en el orden
    position = {task: i for i, task in enumerate(order)}
    
    # Comenzar con un puntaje base de 100
    score = 100
    
    # Iterar sobre cada tarea que tiene restricciones de precedencia
    for task, preds in precedence.items():
        # Para cada predecesor requerido por esta tarea
        for pred in preds:
            # Verificar si el predecesor viene DESPUÉS de la tarea (violación)
            if position[pred] > position[task]:
                # Penalización fuerte por restricción violada
                score -= 30
            else:
                # Restricción satisfecha, pero penalizar distancia entre tareas
                # Calcular cuántas tareas hay entre el predecesor y la tarea dependiente
                # Ejemplo: si pred está en posición 2 y task en posición 5,
                # distancia = 5 - 2 - 1 = 2 tareas en el medio
                # Penalizar con 2 puntos por cada tarea intermedia
                score -= (position[task] - position[pred] - 1) * 2
    
    # Retornar el puntaje final (mayor es mejor)
    return score

# Probar la función de aptitud
print(f"Orden: {valid_order}")
print(f"  Aptitud: {fitness(valid_order)}")
print(f"  Válido: {is_valid(valid_order)}\n")

print(f"Orden: {invalid_order}")
print(f"  Aptitud: {fitness(invalid_order)}")
print(f"  Válido: {is_valid(invalid_order)}")

Orden: ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J']
  Aptitud: 76
  Válido: True

Orden: ['C', 'A', 'B', 'D', 'E', 'F', 'G', 'H', 'I', 'J']
  Aptitud: 48
  Válido: False


## Explicación Detallada: Sistema de Penalizaciones

### Dos Tipos de Penalizaciones:

**1. Penalización por Violación (-30 puntos)**

Esto ocurre cuando una restricción está **rota** - cuando una tarea aparece antes de su prerequisito.

**Ejemplo:**
```python
Orden: ['C', 'A', 'B', 'D', 'E', 'F', 'G', 'H', 'I', 'J']
       Posición: C=0, A=1

Restricción: C requiere A
Problema: C está en posición 0, A está en posición 1
         → A viene DESPUÉS de C (¡violación!)
Penalización: -30 puntos
```

**2. Penalización por Distancia (-2 puntos por tarea en el medio)**

Esto ocurre cuando una restricción está **satisfecha** pero las tareas están lejos.

**Ejemplo:**
```python
Orden: ['A', 'B', 'E', 'G', 'H', 'C', 'D', 'F', 'I', 'J']
       Posición: A=0, C=5

Restricción: C requiere A
Verificar: A (posición 0) viene antes de C (posición 5) ✓ ¡Válido!
Pero: Hay 4 tareas entre ellas (B, E, G, H)
Distancia = position[C] - position[A] - 1 = 5 - 0 - 1 = 4
Penalización: 4 * 2 = -8 puntos
```

### ¿Por Qué Este Diseño?

El sistema de penalización tiene un objetivo inteligente de dos niveles:

1. **Objetivo Primario**: Encontrar una solución válida (sin violaciones)
   - Las violaciones están fuertemente penalizadas (-30)
   - Una solución con violaciones tendrá una aptitud mucho menor

2. **Objetivo Secundario**: Entre soluciones válidas, encontrar la "mejor"
   - Mantener tareas dependientes cerca
   - Minimizar sobrecarga de coordinación en planificación real

### Ejemplo Completo:

Analicemos este orden: `['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J']`

```
Posiciones: A=0, B=1, C=2, D=3, E=4, F=5, G=6, H=7, I=8, J=9

Verificando cada restricción:

C requiere A: 
  - A en 0, C en 2 → Válido ✓
  - Distancia = 2 - 0 - 1 = 1 tarea en medio (B)
  - Penalización: 1 * 2 = -2

D requiere A:
  - A en 0, D en 3 → Válido ✓
  - Distancia = 3 - 0 - 1 = 2 tareas en medio (B, C)
  - Penalización: 2 * 2 = -4

E requiere B:
  - B en 1, E en 4 → Válido ✓
  - Distancia = 4 - 1 - 1 = 2 tareas en medio (C, D)
  - Penalización: 2 * 2 = -4

F requiere C y D:
  - C en 2, F en 5 → Distancia = 5 - 2 - 1 = 2 → Penalización: -4
  - D en 3, F en 5 → Distancia = 5 - 3 - 1 = 1 → Penalización: -2

G requiere E y F:
  - E en 4, G en 6 → Distancia = 6 - 4 - 1 = 1 → Penalización: -2
  - F en 5, G en 6 → Distancia = 6 - 5 - 1 = 0 → Penalización: 0

H requiere G:
  - G en 6, H en 7 → Distancia = 7 - 6 - 1 = 0 → Penalización: 0

I requiere F:
  - F en 5, I en 8 → Distancia = 8 - 5 - 1 = 2 → Penalización: -4

J requiere H e I:
  - H en 7, J en 9 → Distancia = 9 - 7 - 1 = 1 → Penalización: -2
  - I en 8, J en 9 → Distancia = 9 - 8 - 1 = 0 → Penalización: 0

Penalizaciones totales: -2 -4 -4 -4 -2 -2 0 0 -4 -2 0 = -24
Aptitud final: 100 - 24 = 76
```

## Parte 2: Creando la Población Inicial

En los algoritmos genéticos, comenzamos con una **población** de soluciones aleatorias y las evolucionamos con el tiempo.

![DAG](figuras/figura2_histograma_inicial.png)

In [21]:
def create_individual():
    # Crear un individuo aleatorio (permutación aleatoria de todas las tareas)
    # random.sample toma todos los elementos de 'tasks' y los retorna en orden aleatorio
    return random.sample(tasks, len(tasks))

def create_population(size=20):
    # Crear una población inicial de 'size' individuos aleatorios
    # Cada individuo es una posible solución (ordenamiento de tareas)
    return [create_individual() for _ in range(size)]

# Generar la población inicial de 20 individuos
population = create_population()

# Mostrar los primeros 5 individuos de la población con sus características
# enumerate(population[:5], 1) itera sobre los primeros 5 elementos comenzando el índice en 1
for i, ind in enumerate(population[:5], 1):
    # Imprimir: número, orden de tareas, fitness y si es válido
    print(f"{i}. {ind} -> Fitness: {fitness(ind):4d}, Válido: {is_valid(ind)}")

1. ['J', 'C', 'F', 'E', 'I', 'D', 'A', 'B', 'H', 'G'] -> Fitness: -134, Válido: False
2. ['F', 'G', 'C', 'J', 'H', 'A', 'I', 'E', 'B', 'D'] -> Fitness: -130, Válido: False
3. ['F', 'H', 'J', 'C', 'D', 'G', 'B', 'E', 'A', 'I'] -> Fitness: -134, Válido: False
4. ['G', 'A', 'F', 'I', 'H', 'E', 'J', 'B', 'D', 'C'] -> Fitness:  -88, Válido: False
5. ['G', 'C', 'D', 'J', 'F', 'B', 'H', 'I', 'A', 'E'] -> Fitness: -106, Válido: False


## Parte 3: Operadores Genéticos

Los algoritmos genéticos usan tres operadores principales para evolucionar la población:

1. **Selección** - Elegir qué individuos pueden reproducirse
2. **Cruce (Crossover)** - Combinar dos padres para crear descendencia
3. **Mutación** - Introducir cambios aleatorios

### 1. Selección: Selección por Torneo

La selección elige qué individuos pueden "reproducirse" basándose en su aptitud.

**¿Por Qué Selección por Torneo?**
- Simple y eficiente
- Mantiene diversidad en la población
- No requiere ordenar toda la población
- Balancea exploración (muestreo aleatorio) y explotación (elegir el mejor)

In [22]:
def selection(pop):
    # Selección por torneo: elegir 2 individuos aleatorios de la población
    # random.sample(pop, 2) retorna una lista con 2 individuos seleccionados aleatoriamente
    a, b = random.sample(pop, 2)
    
    # Retornar el individuo con mejor fitness (mayor puntaje)
    # Operador ternario: si fitness(a) > fitness(b), retorna 'a', sino retorna 'b'
    return a if fitness(a) > fitness(b) else b

# Demostrar selección
print("Demostración de Selección por Torneo:\n")
for i in range(5):
    selected = selection(population)
    print(f"Seleccionado: {selected} (Aptitud: {fitness(selected)})")

Demostración de Selección por Torneo:

Seleccionado: ['A', 'F', 'J', 'C', 'E', 'B', 'G', 'D', 'H', 'I'] (Aptitud: -92)
Seleccionado: ['G', 'A', 'F', 'I', 'H', 'E', 'J', 'B', 'D', 'C'] (Aptitud: -88)
Seleccionado: ['F', 'G', 'C', 'J', 'H', 'A', 'I', 'E', 'B', 'D'] (Aptitud: -130)
Seleccionado: ['A', 'D', 'H', 'E', 'C', 'I', 'B', 'J', 'F', 'G'] (Aptitud: -34)
Seleccionado: ['H', 'C', 'J', 'B', 'D', 'F', 'A', 'I', 'G', 'E'] (Aptitud: -74)


### 2. Cruce: Cruce de Orden (Order Crossover - OX)

El cruce combina dos soluciones padres para crear una solución hijo.

**¿Por Qué Cruce de Orden?**
- Preserva el ordenamiento relativo de ambos padres
- Garantiza que todas las tareas aparezcan exactamente una vez (permutación válida)
- Hereda subsecuencias beneficiosas de los padres

![Crossover](figuras/crossover.png)


**Ejemplo:**
```
p1 = [A, B, C, D, E, F, G, H, I, J]
p2 = [B, D, A, F, C, E, H, G, J, I]
Si seleccionamos posiciones 2-5 de p1:
  hijo comienza como [_, _, C, D, E, F, _, _, _, _]
  Llenar restantes con orden de p2: [B, A, C, D, E, F, H, G, J, I]
```

In [23]:
def crossover(p1, p2):
    # Obtener el tamaño de los padres (número de tareas)
    size = len(p1)
    
    # Seleccionar dos posiciones aleatorias y ordenarlas para obtener el rango [start, end)
    # Ejemplo: si random.sample retorna [7, 3], sorted lo convierte en [3, 7]
    start, end = sorted(random.sample(range(size), 2))
    
    # Crear un hijo inicializado con None en todas las posiciones
    child = [None]*size
    
    # Copiar el segmento del padre 1 entre start y end al hijo
    # Esto preserva una subsecuencia del padre 1
    child[start:end] = p1[start:end]
    
    # Crear una lista con los elementos del padre 2 que NO están ya en el hijo
    # Esto mantiene el orden relativo de los elementos del padre 2
    fill = [x for x in p2 if x not in child]
    
    # Inicializar índice para recorrer la lista 'fill'
    idx = 0
    
    # Llenar las posiciones None del hijo con elementos de 'fill' en orden
    for i in range(size):
        # Si esta posición aún está vacía (None)
        if child[i] is None:
            # Llenarla con el siguiente elemento de la lista 'fill'
            child[i] = fill[idx]
            # Avanzar al siguiente elemento de 'fill'
            idx += 1
    
    # Retornar el hijo completo (todas las tareas presentes exactamente una vez)
    return child

# Demostrar cruce
p1 = ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J']
p2 = ['B', 'D', 'A', 'F', 'C', 'E', 'H', 'G', 'J', 'I']

print("Demostración de Cruce:\n")
print(f"Padre 1: {p1}")
print(f"Padre 2: {p2}")
print("\nHijos creados por cruce:")
for i in range(5):
    child = crossover(p1, p2)
    print(f"  {i+1}. {child}")

Demostración de Cruce:

Padre 1: ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J']
Padre 2: ['B', 'D', 'A', 'F', 'C', 'E', 'H', 'G', 'J', 'I']

Hijos creados por cruce:
  1. ['B', 'D', 'A', 'C', 'E', 'F', 'G', 'H', 'I', 'J']
  2. ['B', 'D', 'A', 'F', 'C', 'E', 'G', 'H', 'I', 'J']
  3. ['B', 'D', 'C', 'A', 'F', 'E', 'H', 'G', 'J', 'I']
  4. ['B', 'D', 'A', 'C', 'E', 'F', 'G', 'H', 'I', 'J']
  5. ['B', 'A', 'C', 'D', 'F', 'E', 'H', 'G', 'J', 'I']


## Explicación Detallada: Cruce de Orden (OX)

El Cruce de Orden es un cruce especial diseñado para **problemas de permutación** donde cada elemento debe aparecer exactamente una vez.

### Proceso Paso a Paso:

**Ejemplo:**
```
Padre 1: [A, B, C, D, E, F, G, H, I, J]
Padre 2: [B, D, A, F, C, E, H, G, J, I]
```

**Paso 1: Elegir puntos de cruce aleatorios**
```
Digamos que seleccionamos aleatoriamente las posiciones 3 y 7
Inicio = 3, Fin = 7
```

**Paso 2: Copiar subcadena del Padre 1**
```
Padre 1: [A, B, C, |D, E, F, G|, H, I, J]
                    ↓  ↓  ↓  ↓
Hijo:    [_, _, _, |D, E, F, G|, _, _, _]
```

**Paso 3: Crear "lista de relleno" del Padre 2**
```
Padre 2: [B, D, A, F, C, E, H, G, J, I]

Remover elementos ya en el hijo (D, E, F, G):
Lista de relleno: [B, A, C, H, J, I]
```

**Paso 4: Llenar posiciones restantes con orden del Padre 2**
```
Hijo: [_, _, _, D, E, F, G, _, _, _]

Llenar de izquierda a derecha con lista de relleno:
Posición 0: B  → [B, _, _, D, E, F, G, _, _, _]
Posición 1: A  → [B, A, _, D, E, F, G, _, _, _]
Posición 2: C  → [B, A, C, D, E, F, G, _, _, _]
Posición 7: H  → [B, A, C, D, E, F, G, H, _, _]
Posición 8: J  → [B, A, C, D, E, F, G, H, J, _]
Posición 9: I  → [B, A, C, D, E, F, G, H, J, I]

Hijo Final: [B, A, C, D, E, F, G, H, J, I]
```

### Representación Visual:

```
Padre 1: [A  B  C |D  E  F  G| H  I  J]
Padre 2: [B  D  A |F  C  E  H| G  J  I]
                    ↓           
          Copiar este segmento de P1
                    ↓
Hijo:    [?  ?  ? |D  E  F  G| ?  ?  ?]
                    ↓
          Llenar con elementos restantes de P2 en orden
                    ↓
Hijo:    [B  A  C |D  E  F  G| H  J  I]
```

### ¿Por Qué Cruce de Orden?

**1. Preserva Validez de Permutación**
- Cada tarea aparece exactamente una vez
- Sin duplicados, sin tareas faltantes

**2. Hereda Bloques de Construcción**
- El segmento del Padre 1 (D-E-F-G) se preserva
- Si esta es una buena subsecuencia, se pasa al hijo

**3. Mantiene Orden Relativo**
- Los elementos del Padre 2 mantienen su orden relativo
- Ejemplo: En P2, B viene antes de A, y esto se preserva en el hijo

### Comparación con Cruce Simple:

**Cruce Simple (no funciona para permutaciones):**
```
Padre 1: [A, B, C, D, E, F, G, H, I, J]
Padre 2: [B, D, A, F, C, E, H, G, J, I]
Cortar en posición 5:

Hijo: [A, B, C, D, E] + [E, H, G, J, I]
    = [A, B, C, D, E, E, H, G, J, I]  ❌ ¡E aparece dos veces! ¡Falta F!
```

**Cruce de Orden (¡funciona!):**
```
Hijo: [B, A, C, D, E, F, G, H, J, I]  ✓ Todas las tareas aparecen exactamente una vez
```

### Desglose del Código:

```python
def crossover(p1, p2):
    size = len(p1)  # = 10
    
    # Seleccionar aleatoriamente dos posiciones y ordenarlas
    # Ejemplo: random.sample da [7, 3] → sorted da [3, 7]
    start, end = sorted(random.sample(range(size), 2))
    # start = 3, end = 7
    
    # Crear hijo lleno con None
    child = [None] * size
    # child = [None, None, None, None, None, None, None, None, None, None]
    
    # Copiar segmento del padre 1
    child[start:end] = p1[start:end]
    # child = [None, None, None, D, E, F, G, None, None, None]
    
    # Obtener elementos de p2 que aún no están en el hijo
    fill = [x for x in p2 if x not in child]
    # p2 = [B, D, A, F, C, E, H, G, J, I]
    # Ya en el hijo: D, E, F, G
    # fill = [B, A, C, H, J, I]
    
    # Llenar posiciones restantes
    idx = 0  # Índice en lista fill
    for i in range(size):  # i va de 0 a 9
        if child[i] is None:
            child[i] = fill[idx]
            idx += 1
    
    # i=0: child[0] es None → child[0] = B, idx=1
    # i=1: child[1] es None → child[1] = A, idx=2
    # i=2: child[2] es None → child[2] = C, idx=3
    # i=3: child[3] es D → saltar
    # i=4: child[4] es E → saltar
    # i=5: child[5] es F → saltar
    # i=6: child[6] es G → saltar
    # i=7: child[7] es None → child[7] = H, idx=4
    # i=8: child[8] es None → child[8] = J, idx=5
    # i=9: child[9] es None → child[9] = I, idx=6
    
    return child
    # Devuelve: [B, A, C, D, E, F, G, H, J, I]
```

### 3. Mutación: Mutación por Intercambio

La mutación introduce cambios aleatorios para mantener la diversidad.

**¿Por Qué Mutación por Intercambio?**
- Simple y preserva validez de permutación
- Tasa de mutación del 20% balancea exploración y explotación
- Ayuda a escapar de óptimos locales

In [24]:
def mutate(ind, prob=0.2):
    # Generar un número aleatorio entre 0 y 1 y compararlo con la probabilidad de mutación
    # Si el número es menor que prob (20% por defecto), ocurre la mutación
    if random.random() < prob:
        # Seleccionar dos posiciones aleatorias diferentes en el individuo
        # random.sample garantiza que i y j sean distintos
        i, j = random.sample(range(len(ind)), 2)
        
        # Intercambiar los elementos en las posiciones i y j (swap)
        # Esto modifica el individuo directamente (mutación in-place)
        ind[i], ind[j] = ind[j], ind[i]
    
    # Retornar el individuo (mutado o no, dependiendo de si se cumplió la condición)
    return ind

# Demostrar mutación
original = ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J']
print("Demostración de Mutación (con tasa de mutación del 100% para demostración):\n")
print(f"Original: {original}")
print("\nVersiones mutadas:")
for i in range(5):
    mutated = original.copy()
    mutate(mutated, prob=1.0)  # 100% mutación para demo
    print(f"  {i+1}. {mutated}")

Demostración de Mutación (con tasa de mutación del 100% para demostración):

Original: ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J']

Versiones mutadas:
  1. ['A', 'B', 'C', 'D', 'H', 'F', 'G', 'E', 'I', 'J']
  2. ['A', 'B', 'C', 'D', 'I', 'F', 'G', 'H', 'E', 'J']
  3. ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'J', 'I']
  4. ['D', 'B', 'C', 'A', 'E', 'F', 'G', 'H', 'I', 'J']
  5. ['B', 'A', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J']


## Explicación Detallada: Proceso de Mutación

La mutación es el operador genético final que introduce cambios aleatorios para mantener la diversidad en la población y ayudar a escapar de óptimos locales.

### El Método de Mutación por Intercambio

```python
def mutate(ind, prob=0.2):
    if random.random() < prob:
        i, j = random.sample(range(len(ind)), 2)
        ind[i], ind[j] = ind[j], ind[i]
    return ind
```

### Proceso Paso a Paso:

**Paso 1: Decidir si ocurre la mutación**
```python
if random.random() < prob:
```
- `random.random()` genera un número entre 0.0 y 1.0
- `prob=0.2` significa 20% de probabilidad de mutación
- Ejemplos:
  - Si random.random() = 0.15 → 0.15 < 0.2 → VERDADERO → ¡Mutar!
  - Si random.random() = 0.87 → 0.87 < 0.2 → FALSO → No mutar
  - Si random.random() = 0.19 → 0.19 < 0.2 → VERDADERO → ¡Mutar!

**Paso 2: Seleccionar dos posiciones aleatorias**
```python
i, j = random.sample(range(len(ind)), 2)
```
Ejemplo con individuo: `[A, B, C, D, E, F, G, H, I, J]`
- `range(10)` = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
- `random.sample` elige 2 posiciones diferentes
- Digamos que elige: `i=2, j=7`

**Paso 3: Intercambiar los elementos**
```python
ind[i], ind[j] = ind[j], ind[i]
```
Antes: `[A, B, C, D, E, F, G, H, I, J]`
       Posición: 0  1  2  3  4  5  6  7  8  9
                       ↑              ↑
                     i=2            j=7

Después: `[A, B, H, D, E, F, G, C, I, J]`
         Posición: 0  1  2  3  4  5  6  7  8  9
                       ↑              ↑
                       H intercambiada con C

### Ejemplo Completo de Seguimiento

Sigamos varios intentos de mutación:

**Individuo:** `[A, B, C, D, E, F, G, H, I, J]`

**Intento 1:**
```
random.random() = 0.73
0.73 < 0.2? NO
Resultado: Sin mutación
Salida: [A, B, C, D, E, F, G, H, I, J]  (sin cambios)
```

**Intento 2:**
```
random.random() = 0.08
0.08 < 0.2? SÍ → ¡Mutar!
Posiciones aleatorias: i=1, j=8

Antes: [A, B, C, D, E, F, G, H, I, J]
           ↑                    ↑
         i=1                  j=8

Intercambiar B e I:
Después: [A, I, C, D, E, F, G, H, B, J]
            ↑                    ↑
```

**Intento 3:**
```
random.random() = 0.15
0.15 < 0.2? SÍ → ¡Mutar!
Posiciones aleatorias: i=0, j=9

Antes: [A, B, C, D, E, F, G, H, I, J]
        ↑                          ↑
      i=0                        j=9

Intercambiar A y J:
Después: [J, B, C, D, E, F, G, H, I, A]
         ↑                          ↑
```

### ¿Por Qué Mutación por Intercambio?

**1. Mantiene Validez de Permutación**
```
Antes: [A, B, C, D, E, F, G, H, I, J]  ✓ Todas las tareas presentes una vez
Después: [A, B, H, D, E, F, G, C, I, J]  ✓ Todas las tareas aún presentes una vez

No como esto (malo):
Después: [A, B, K, D, E, F, G, C, I, J]  ✗ ¡K no es una tarea válida!
```

**2. Cambios Pequeños y Locales**
- Solo 2 posiciones cambian
- No destruye completamente las buenas soluciones
- Exploración gradual

**3. Simple y Efectiva**
- Fácil de implementar
- Computacionalmente económica
- Probada para funcionar bien en problemas de permutación

### Compromisos de la Probabilidad de Mutación

**Tasa de Mutación Baja (ej., 0.05 = 5%)**
```
Ventajas:
- Preserva buenas soluciones por más tiempo
- Más explotación de áreas buenas actuales

Desventajas:
- Puede quedarse atascado en óptimos locales
- Menos exploración
- Puede converger demasiado rápido

Ejemplo: De 20 hijos, solo ~1 se muta
```

**Tasa de Mutación Media (ej., 0.2 = 20%)**
```
Ventajas:
- Buen balance de exploración y explotación
- Mantiene diversidad
- Estándar para muchos problemas

Desventajas:
- Puede ralentizar ligeramente la convergencia

Ejemplo: De 20 hijos, ~4 se mutan
```

**Tasa de Mutación Alta (ej., 0.5 = 50%)**
```
Ventajas:
- Máxima exploración
- Difícil quedarse atascado
- Buena para paisajes muy difíciles

Desventajas:
- Puede interrumpir buenas soluciones con demasiada frecuencia
- Puede comportarse casi como búsqueda aleatoria
- Lenta para converger

Ejemplo: De 20 hijos, ~10 se mutan
```

### Impacto Real en Nuestro Problema

Veamos cómo ayuda la mutación:

**Escenario: La población convergió a una solución subóptima**

```
La mayoría de los individuos se ven así:
[A, C, D, B, E, F, G, I, H, J]
Aptitud: 78 (válida pero no óptima)
```

**Sin mutación:**
- El cruce solo recombina estos individuos similares
- Crea descendencia casi idéntica
- Población atascada en aptitud 78
- No puede escapar de este óptimo local

**Con mutación (tasa 20%):**
```
Original: [A, C, D, B, E, F, G, I, H, J]
Mutar posiciones 1 y 3:

Después: [A, B, D, C, E, F, G, I, H, J]
Aptitud: 80 (¡mejor!)

Este nuevo individuo ahora puede:
1. Ser seleccionado más (mayor aptitud)
2. Propagarse mediante cruce
3. Llevar la población a mejores soluciones
```

### Mutación en Acción a Través de Generaciones

Mostremos una secuencia realista:

**Generación 1:** Población aleatoria
```
Individuo: [J, F, C, E, H, G, I, A, D, B]
Aptitud: -126 (terrible, muchas violaciones)
```

**Después del cruce:**
```
Hijo: [A, F, C, D, E, H, G, I, B, J]
Aptitud: -48 (mejor, menos violaciones)
```

**Luego mutación (si ocurre):**
```
Intercambiar posiciones 5 y 8:
Mutado: [A, F, C, D, E, B, G, I, H, J]
Aptitud: -36 (¡aún mejor! mutación afortunada)
```

**Generación 14:** Población mejorando
```
Individuo: [A, C, D, B, E, H, F, G, I, J]
Aptitud: 42 (¡cerca de ser válida!)
```

**Después del cruce:**
```
Hijo: [A, C, D, B, E, F, H, G, I, J]
Aptitud: 74 (válida pero H antes de G está mal)
```

**Luego mutación:**
```
Intercambiar posiciones 5 y 6:
Antes: [A, C, D, B, E, F, H, G, I, J]
                      ↑  ↑
Después: [A, C, D, B, E, H, F, G, I, J]
Aptitud: 42 (¡peor! Mala mutación)

Este individuo probablemente no será seleccionado
Pero está bien - la diversidad se mantiene
```

### El Código en Detalle

```python
def mutate(ind, prob=0.2):
    # ind: El individuo a potencialmente mutar
    # prob: Probabilidad de mutación (por defecto 20%)
    
    # Generar número aleatorio entre 0 y 1
    random_value = random.random()
    # Ejemplos: 0.15, 0.87, 0.42, 0.09, etc.
    
    # Verificar si debemos mutar (20% de probabilidad)
    if random_value < prob:
        # ¡SÍ! Ocurre mutación
        
        # Seleccionar 2 posiciones aleatorias diferentes
        # Ejemplo: si len(ind) = 10
        # range(10) = [0,1,2,3,4,5,6,7,8,9]
        # random.sample elige 2: quizás [3, 7]
        i, j = random.sample(range(len(ind)), 2)
        
        # Intercambiar elementos en posiciones i y j
        # Asignación simultánea de Python
        # Ejemplo:
        #   ind = [A, B, C, D, E, F, G, H, I, J]
        #   i=3 (D), j=7 (H)
        #   Después: [A, B, C, H, E, F, G, D, I, J]
        ind[i], ind[j] = ind[j], ind[i]
    # else: Sin mutación, individuo sin cambios
    
    # Devolver el individuo (mutado o no)
    return ind
```

### ¿Por Qué No Otros Métodos de Mutación?

**1. Mutación por Inserción** (remover y reinsertar)
```
Antes: [A, B, C, D, E, F, G, H, I, J]
Remover C, insertar después de G:
Después: [A, B, D, E, F, G, C, H, I, J]

Problema: Más compleja, efecto similar al intercambio
```

**2. Mutación por Inversión** (invertir un segmento)
```
Antes: [A, B, C, D, E, F, G, H, I, J]
Invertir posiciones 2-6:
Después: [A, B, G, F, E, D, C, H, I, J]

Problema: Demasiado disruptiva para este problema
```

**3. Mutación por Mezcla** (mezclar aleatoriamente un segmento)
```
Antes: [A, B, C, D, E, F, G, H, I, J]
Mezclar posiciones 3-7:
Después: [A, B, C, G, F, E, D, H, I, J]

Problema: Muy disruptiva, difícil mantener buenos patrones
```

**La mutación por intercambio es mejor aquí porque:**
- Disrupción mínima
- Rápida de calcular
- Aún explora efectivamente
- Mantiene la propiedad de permutación automáticamente

### Ejemplo Interactivo

Mostremos qué sucede con diferentes tasas de mutación en el mismo individuo:

**Original:** `[A, B, C, D, E, F, G, H, I, J]`

**Con 0% mutación (sin mutación):**
```
10 intentos → Todos devuelven: [A, B, C, D, E, F, G, H, I, J]
Sin diversidad añadida
```

**Con 20% mutación:**
```
Intento 1: Sin mutación → [A, B, C, D, E, F, G, H, I, J]
Intento 2: Mutar (3,8) → [A, B, C, I, E, F, G, H, D, J]
Intento 3: Sin mutación → [A, B, C, D, E, F, G, H, I, J]
Intento 4: Sin mutación → [A, B, C, D, E, F, G, H, I, J]
Intento 5: Mutar (1,6) → [A, G, C, D, E, F, B, H, I, J]
Intento 6: Sin mutación → [A, B, C, D, E, F, G, H, I, J]
...
Aproximadamente 2 de cada 10 se mutan
```

**Con 100% mutación (siempre mutar):**
```
Intento 1: Mutar (0,5) → [F, B, C, D, E, A, G, H, I, J]
Intento 2: Mutar (2,9) → [A, B, J, D, E, F, G, H, I, C]
Intento 3: Mutar (4,7) → [A, B, C, D, H, F, G, E, I, J]
Intento 4: Mutar (1,3) → [A, D, C, B, E, F, G, H, I, J]
...
Cada uno se muta
```

## Parte 4: Proceso de Evolución

Ahora combinamos todo para evolucionar nuestra población:

![evolucion](figuras/figura4_evolucion_fitness.png)


Cada generación sigue este proceso:
1. Seleccionar padres basándose en aptitud (selección por torneo)
2. Crear hijos mediante cruce
3. Aplicar mutación
4. Reemplazar población antigua con nueva

In [25]:
def evolve(population, generations=50):
    # Iterar a través de cada generación (50 por defecto)
    for gen in range(generations):
        # Crear una lista vacía para la nueva población
        new_pop = []
        
        # Generar tantos hijos como individuos hay en la población actual
        for _ in range(len(population)):
            # Seleccionar el primer padre usando selección por torneo
            p1 = selection(population)
            
            # Seleccionar el segundo padre usando selección por torneo
            p2 = selection(population)
            
            # Crear un hijo combinando los dos padres mediante cruce (crossover)
            child = crossover(p1, p2)
            
            # Aplicar mutación al hijo con probabilidad del 20%
            child = mutate(child)
            
            # Añadir el hijo a la nueva población
            new_pop.append(child)
        
        # Reemplazar la población antigua con la nueva población
        population = new_pop
        
        # Encontrar el mejor individuo de la generación actual
        # max() con key=fitness encuentra el individuo con mayor fitness
        best = max(population, key=fitness)
        
        # Imprimir información de la generación: número, mejor individuo, fitness y validez
        print(f"Gen {gen+1:2d}: {best} | Fitness: {fitness(best):4d} | Válido: {is_valid(best)}")
    
    # Retornar el mejor individuo de la última generación
    return best

## Ejecutar el Algoritmo

In [30]:
population = create_population()
best = evolve(population)
print(f"\nMejor solución: {best}")
print(f"Fitness: {fitness(best)}")
print(f"Válida: {is_valid(best)}")

Gen  1: ['D', 'A', 'E', 'C', 'F', 'I', 'B', 'G', 'H', 'J'] | Fitness:   14 | Válido: False
Gen  2: ['D', 'F', 'H', 'B', 'E', 'G', 'I', 'A', 'C', 'J'] | Fitness:  -20 | Válido: False
Gen  3: ['H', 'B', 'A', 'C', 'D', 'F', 'I', 'J', 'E', 'G'] | Fitness:   36 | Válido: False
Gen  4: ['D', 'B', 'E', 'F', 'A', 'G', 'I', 'H', 'C', 'J'] | Fitness:   12 | Válido: False
Gen  5: ['H', 'B', 'A', 'C', 'D', 'F', 'E', 'G', 'I', 'J'] | Fitness:   36 | Válido: False
Gen  6: ['H', 'B', 'A', 'C', 'D', 'E', 'F', 'G', 'I', 'J'] | Fitness:   36 | Válido: False
Gen  7: ['H', 'B', 'A', 'C', 'D', 'E', 'F', 'G', 'I', 'J'] | Fitness:   36 | Válido: False
Gen  8: ['B', 'H', 'A', 'C', 'D', 'E', 'F', 'G', 'I', 'J'] | Fitness:   36 | Válido: False
Gen  9: ['B', 'A', 'E', 'G', 'H', 'D', 'C', 'F', 'I', 'J'] | Fitness:   44 | Válido: False
Gen 10: ['H', 'B', 'A', 'C', 'D', 'F', 'E', 'G', 'I', 'J'] | Fitness:   36 | Válido: False
Gen 11: ['H', 'A', 'C', 'D', 'B', 'F', 'E', 'G', 'I', 'J'] | Fitness:   38 | Válido: False